In [1]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(".").absolute().parent))
sys.path.append(str(Path(".").absolute().parent) + "/src/pipeline/pipeline_utils")
import pipeline_utils 
import argparse
import datetime
import pytz

In [2]:
print(datetime.datetime.now(pytz.timezone("US/Central")).strftime("%Y-%m-%d %H:%M:%S"))


2024-12-03 08:02:44


In [3]:
try:
    args = pipeline_utils.MarkdownArgs().get_args()
except:
    parser = argparse.ArgumentParser()
    parser.add_argument("--COMMIT_ID", required=True, type=str)
    parser.add_argument("--BRANCH", required=True, type=str)
    parser.add_argument("--is_prod", required=False, type=lambda x: (str(x).lower() == 'true'))
    sys.args = [
        "--COMMIT_ID", "1234",
        "--BRANCH", "dev",
        "--is_prod", "False",
    ]
    args = parser.parse_args(sys.args)

usage: ipykernel_launcher.py [-h] --COMMIT_ID COMMIT_ID --BRANCH BRANCH
                             [--is_prod IS_PROD]
ipykernel_launcher.py: error: the following arguments are required: --COMMIT_ID, --BRANCH


In [4]:
PARAMS = pipeline_utils.YamlImport("settings_v1.yml").yaml_import()
BRANCH_ID = args.BRANCH
is_prod = args.is_prod

In [5]:
if BRANCH_ID == "stage" and is_prod == True:
    BRANCH_ID = "prod"
    
ENV = BRANCH_ID
ENV


'dev'

In [6]:
PROJECT_ID = PARAMS['envs'][ENV]['PROJECT_ID']
BASE_IMAGE = PARAMS['envs'][ENV]['BASE_IMAGE']

PIPELINE_ROOT = PARAMS['envs'][ENV]['PIPELINE_ROOT']

print(f"""
PROJECT_ID: {PROJECT_ID}, 
\nBASE_IMAGE: {BASE_IMAGE},
\nPIPELINE_ROOT: {PIPELINE_ROOT}, 
""")


PROJECT_ID: wmt-mlp-p-price-npd-pricing, 

BASE_IMAGE: gcr.io/wmt-mlp-p-price-npd-pricing/markdown-training-pipeline-dev:latest,

PIPELINE_ROOT: gs://md-training-pipeline-bucket-nonprod, 



In [7]:
BASE_IMAGE

'gcr.io/wmt-mlp-p-price-npd-pricing/markdown-training-pipeline-dev:latest'

In [8]:
%%capture
!docker build -t {BASE_IMAGE} . -f ./base_image/Dockerfile --progress=plain --no-cache


In [9]:
!docker push {BASE_IMAGE}


The push refers to repository [gcr.io/wmt-mlp-p-price-npd-pricing/markdown-training-pipeline-dev]

0afaf0ea: Preparing 
9915e0de: Preparing 
c241803b: Preparing 
1b69a69f: Preparing 
8e860a94: Preparing 
7ac8a7d5: Preparing 
c2c8ff37: Preparing 
dc44ddd4: Preparing 
da64173e: Preparing 
0cec99b1: Preparing 
5dbccd29: Preparing 
919c0dbd: Preparing 
055ab182: Preparing 
2dbd502b: Preparing 
ad1bf5f7: Preparing 
cfd5102e: Preparing 
659bc1fa: Preparing 
cb5563d6: Preparing 
03addca4: Preparing 
f4751a42: Preparing 
915e0de: Pushed   1.951GB/1.911GBlatest: digest: sha256:7f72429096894ac3615ca0ddf417c8c184b1c30391dc9f3a6b4dbdc20b604970 size: 4718


## to maintain the boot disk without memory leak. 


In [10]:
!docker image prune -f


Deleted Images:
untagged: gcr.io/wmt-mlp-p-price-npd-pricing/markdown-training-pipeline-dev@sha256:4c9b8331acc056f2c9ea19399e48ae5f1b8518f8ffbbed6801f1df7f21721126
deleted: sha256:d94e9bfd3f65629c1864fa02043f20cefad07418164f2cd415fb647ced7d88d7
deleted: sha256:d9b78a9ef4f3caa0df9f84743b0412d9c4f4ea4027a96b06950058b6eb7ce344
deleted: sha256:7d97ae47497c2981666d5a20e6ef43324f200d61cbfdcacf0b39eeb8ed782ef2
deleted: sha256:bb3fe3006ae7a3e7cbf9c08c535368f6c68833658fc177dc96a8391df8059e69
deleted: sha256:9aee0b72b246e09dd3f0104c82a85841f0917dab8759ce59c1c0beaa4159c49c
deleted: sha256:5e0ee61eb88d03067b1b5fbb1f0444324ef35bad70ff7105a241d22a0829c2b9
deleted: sha256:4d41dbb2aabc6d25205518d2297d00bbf848445e23ea228805b2120298000940
deleted: sha256:ef82cc06322bbbbdd47ee490d604a3e521037b02a394ae2dcaaff73832f01319
deleted: sha256:d96726098f967a3e01c5d475366a254942f60219bdc5cc4056f04f5ce527760c
deleted: sha256:55fe0845104006527284d7e101ee218a95003672073532c65ad7428a258cb0b4
deleted: sha256:da86d38b53